In [1]:
import warnings
from itertools import chain, combinations, product
import numpy as np
import numpy.typing as npt
from typing import Generator, Any, Iterator, Tuple, Union
import pandas as pd
import time

In [2]:
def powerset(n: int) -> Iterator[Tuple[Any, ...]]:
    """
    A power set of range(n)

    Based on recipe from python itertools documentation:

    https://docs.python.org/2/library/itertools.html#recipes

    Parameters
    ----------
    n : int
        The defining parameter of the powerset.

    Returns
    -------
    Iterator
        The powerset
    """
    return chain.from_iterable(combinations(range(n), r) for r in range(n + 1))

In [4]:
def potential_support_pairs(A: npt.NDArray, B: npt.NDArray, non_degenerate: bool = False) -> Generator[tuple, Any, None]:
    """
    A generator for the potential support pairs


    Parameters
    ----------
    A : array
        The row player utility matrix.
    B : array
        The column player utility matrix
    non_degenerate : bool
        Whether or not to consider supports of equal size. By default
        (False) only considers supports of equal size.

    Yields
    -------
    Generator
        A pair of possible supports.
    """
    p1_num_strategies, p2_num_strategies = A.shape
    for support1 in (s for s in powerset(p1_num_strategies) if len(s) > 0):
        for support2 in (
            s 
            for s in powerset(p2_num_strategies) 
            if (len(s) > 0 and non_degenerate) or len(s) == len(support1)
        ):
            
            yield support1, support2

In [3]:
def solve_indifference(A, rows=None, columns=None) -> Union[bool, Any]:
    """
    Solve the indifference for a payoff matrix assuming support for the
    strategies given by columns

    Finds vector of probabilities that makes player indifferent between
    rows.  (So finds probability vector for corresponding column player)

    Parameters
    ----------
    A : array
        The row player utility matrix.
    rows : array
        Array of integers corresponding to rows to consider.
    columns : array
        Array of integers corresponding to columns to consider.

    Returns
    -------
    Union
        The solution to the indifference equations.
    """
    # Ensure differences between pairs of pure strategies are the same
    M = (A[np.array(rows)] - np.roll(A[np.array(rows)], 1, axis=0))[:-1]
    # Columns that must be played with prob 0
    zero_columns = set(range(A.shape[1])) - set(columns)

    if zero_columns != set():
        M = np.append(
            M,
            [[int(i == j) for i, col in enumerate(M.T)] for j in zero_columns],
            axis=0,
        )

    # Ensure have probability vector
    M = np.append(M, np.ones((1, M.shape[1])), axis=0)
    b = np.append(np.zeros(len(M) - 1), [1])

    try:
        prob = np.linalg.solve(M, b)
        if all(prob >= 0):
            return prob
        return False
    except np.linalg.LinAlgError:
        return False

In [29]:
def indifference_strategies(A: npt.NDArray, B: npt.NDArray, pair, non_degenerate: bool = False, tol: float = 10**-16) ->Tuple[ Any, Any]:
    """
    A function that returns a pair of strategies that consitutes a completely mixed Nash equilibrium of the block game
    
    Parameters
    ----------
    A : array
        The row player utility matrix.
    B : array
        The column player utility matrix
    pair : tuple
        a 2-tuple of numpy arrays of integers.
    non_degenerate : bool
        Whether or not to consider supports of equal size. By default
        (False) only considers supports of equal size.
    tol : float
        A tolerance parameter for equality.

    Yields
    ------
    Return
        A a pair of strategies that are indifferent on each
        potential support. Return False if they are not valid (not a
        probability vector OR not fully on the given support).
    """
    if non_degenerate:
        tol = min(tol, 0)

    s1 = solve_indifference(B.T, *(pair[::-1]))
    s2 = solve_indifference(A, *pair)

    if obey_support(s1, pair[0], tol=tol) and obey_support(s2, pair[1], tol=tol):
        return s1, s2
    else:
        return None, None

In [6]:
def obey_support(strategy, support: npt.NDArray, tol: float = 10**-16) -> bool:
    """
    Test if a strategy obeys its support

    Parameters
    ----------
    strategy: array
        A given strategy vector
    support: array
        A strategy support
    tol : float
        A tolerance parameter for equality.

    Returns
    -------
    bool
        whether or not that strategy does indeed have the given support
    """
    if strategy is False:
        return False
    if not all(
        (i in support and value > tol) or (i not in support and value <= tol)
        for i, value in enumerate(strategy)
    ):
        return False
    return True

In [30]:
def is_ne_ind(strategy_pair: tuple, support_pair: Tuple[npt.NDArray, npt.NDArray], payoff_matrices: Tuple[npt.NDArray, npt.NDArray],) -> (bool, int):
    """
    Test if a given strategy pair is a pair of best responses and if true calculates its index

    Parameters
    ----------
    strategy_pair: tuple
        a 2-tuple of numpy arrays.
    support_pair: tuple
        a 2-tuple of numpy arrays of integers.
    payoff_matrices: tuple
        a 2-tuple of numpy array of payoff matrices.

    Returns
    -------
    bool
        True if a given strategy pair is a pair of best responses.
    int
        If pair is consitutes a Nash equilibrium, return its index.
    """
    if strategy_pair[0] is None:
        return (bool(0), 0)
    
    A, B = payoff_matrices
    # Payoff against opponents strategies:
    u = strategy_pair[1].reshape(strategy_pair[1].size, 1)
    row_payoffs = np.dot(A, u)

    v = strategy_pair[0].reshape(strategy_pair[0].size, 1)
    column_payoffs = np.dot(B.T, v)

    # Pure payoffs on current support:
    row_support_payoffs = row_payoffs[np.array(support_pair[0])]
    column_support_payoffs = column_payoffs[np.array(support_pair[1])]

    A_sup = A[np.array(support_pair[0])].T [np.array(support_pair[1])].T
    B_sup = B[np.array(support_pair[0])].T [np.array(support_pair[1])].T
    
    if row_payoffs.max() == row_support_payoffs.max() and column_payoffs.max() == column_support_payoffs.max():
        index = np.sign((-1)**(1+len(support_pair[0]))*np.linalg.det(A_sup)*np.linalg.det(B_sup)) # Govindan and Wilson (1997)
        return (bool(1), index)
    else: 
        return (bool(0), 0)
    return 

In [47]:
def calc_index_block(blocks_w_index, pair, ne_index) -> Union[Tuple[Any, Any], int, Any]:
    """
    Tests if a given block contains a block from a given set of blocks

    Parameters
    ----------
    pair: tuple
        a 2-tuple of numpy arrays of integers.
    min_gbs: array
        a numpy array of 2-tuple of numpy arrays of integers.
    ne_index: tuple
        a pair of strategy profiles with their index

    Returns
    -------
    tuple: To be simplified and described
 
    """
    gb = bool(0)
    temp = 0
    ne_indices = []
    for block, ne_index_b in blocks_w_index:
        if set(block[0][0]) <= set(pair[0]) and set(block[0][1]) <= set(pair[1]):
            temp =+ block[1]
            if ne_index_b[0] is not None:
                ne_indices.append(ne_index_b)
    index = ne_index[1] + temp
    ne_indices.append(ne_index)
    if index==1:
        gb=bool(1)
    return [pair, ne_index[1]], ne_indices, gb 

In [44]:
def is_nested(pair, min_gbs) -> bool:
    """
    Tests if a given block contains a block from a given set of blocks

    Parameters
    ----------
    pair: tuple
        a 2-tuple of numpy arrays of integers.
    min_gbs: array
        a numpy array of 2-tuple of numpy arrays of integers.

    Returns
    -------
    bool
        True if a given block properly contains a block from the given set.
    """
    for mgb in min_gbs:
            if set(mgb[0]) <= set(pair[0]) and set(mgb[1]) <= set(pair[1]):
                return bool(1)
    return bool(0)

In [16]:
def support_enumeration(A: npt.NDArray, B: npt.NDArray, non_degenerate: bool = False, tol: float = 10**-16):# -> Generator[Tuple[bool, bool], Any, None]:
    """
    Obtain the Nash equilibria using support enumeration.

    Algorithm implemented here is a modified version of Algorithm 3.4 of [Nisan2007] that gives minimal game blocks and solid outcomes 

    See algorithm described elsewhere

    Parameters
    ----------
    A : array
        The row player utility matrix.
    B : array
        The column player utility matrix
    non_degenerate : bool
        Whether or not to consider supports of equal size. By default
        (False) only considers supports of equal size.
    tol : float
        A tolerance parameter for equality.

    Yields
    -------
    Generator
        The minimal game block with its combined index and the equilibria with their corresponding indices.
    """
    blocks_w_index = []
    min_gameb = []
    solid_outcomes = []
    for pair in potential_support_pairs(A, B, non_degenerate=non_degenerate):
        if is_nested(pair, min_gameb):
            continue
        else:
            s1, s2 = indifference_strategies(A, B, pair, non_degenerate=non_degenerate, tol=tol)
            ne, index = is_ne_ind((s1, s2), pair, (A, B))
            if ne:
                ne_index = [s1,s2], index
            else:
                ne_index = [None, 0]
            block_w_index, ne_indices, gb = calc_index_block(blocks_w_index, pair, ne_index)
            blocks_w_index.append([block_w_index, ne_index])
            if gb:
                min_gameb.append(pair)
                yield block_w_index[0], ne_indices[0]

In [52]:
m, n = np.random.rand(5,5), np.random.rand(5,5)
print(m)
print(n)
for i, j in support_enumeration(m,n):
    print("block:", i, "NE:", j[0], "index:", j[1])

[[0.31470082 0.31621274 0.7628385  0.70884285 0.29368735]
 [0.43446093 0.2313267  0.58232856 0.68021306 0.14821953]
 [0.08936838 0.24902682 0.26011335 0.42299947 0.69500044]
 [0.77848303 0.92778289 0.26487824 0.03698962 0.6641087 ]
 [0.07981482 0.64286412 0.76703548 0.83456927 0.60313477]]
[[0.60545001 0.55795111 0.0196754  0.75590621 0.24393973]
 [0.74056941 0.62750744 0.18701071 0.31953603 0.2628513 ]
 [0.16483105 0.71571907 0.28823627 0.28275728 0.52879504]
 [0.82734265 0.36212731 0.31374401 0.73361349 0.37413965]
 [0.90760232 0.98852551 0.0247682  0.42758515 0.10557889]]
block: ((3,), (0,)) NE: [array([0., 0., 0., 1., 0.]), array([1., 0., 0., 0., 0.])] index: 1.0


In [51]:
print(save)
print(it)
for i, j in support_enumeration(save,it):
    print("block:", i, "NE:", j[0], "index:", j[1])

[[0.23829679 0.8745105  0.75425932 0.55634938]
 [0.05582288 0.1056687  0.91071992 0.5043406 ]
 [0.98741089 0.21975027 0.90555963 0.53885173]
 [0.92343284 0.58710128 0.64017278 0.58807912]]
[[0.47666705 0.803487   0.67365332 0.16357639]
 [0.39155968 0.03095863 0.74201943 0.40907269]
 [0.3773084  0.65782603 0.15010643 0.49308804]
 [0.18742591 0.0201202  0.36952566 0.24705012]]
block: ((0,), (1,)) NE: [array([1., 0., 0., 0.]), array([0., 1., 0., 0.])] index: 1.0
block: ((1,), (2,)) NE: [array([0., 1., 0., 0.]), array([0., 0., 1., 0.])] index: 1.0
block: ((2, 3), (2, 3)) NE: [array([0.        , 0.        , 0.26312957, 0.73687043]), array([0.        , 0.        , 0.15646905, 0.84353095])] index: 1.0


In [ ]:
n = 10000
df2 = []
for j in range(6):
    df = []
    for k in range(6):
        n_ne = 0
        r, c = j + 2, k + 2
        for i in range(n):
            l, m = np.random.rand(r,c), np.random.rand(r,c)
            for _, _ in enumerate(support_enumeration(l,m)):
                n_ne +=  1
        df.append(n_ne / n)
    df2.append(df)
    print(df2)

In [ ]:
n = 100000

n_ne = 0
tic = time.perf_counter()
for i in range(n):

    l, m = np.random.rand(2,2), np.random.rand(2,2)
    for _, _ in enumerate(support_enumeration(l,m)):
        n_ne +=  1
    if i%500==0:
        toc = time.perf_counter()
        print(np.round(toc -tic,1) , i / n)
        
n_ne / n

In [58]:
a = [[1.1269, 1.183, 1.2243, 1.2492, 1.2788, 1.2965], [1.1824, 1.2859, 1.3613, 1.4135, 1.4672, 1.5115], [1.2294, 1.3579, 1.4684, 1.5566, 1.6217, 1.6898], 
     [1.2524, 1.4258, 1.5346, 1.6621, 1.7622, 1.8395], [1.2841, 1.4762, 1.616, 1.7557, 1.8743, 2.0174 ], ["?", "?", "?", "?", "?", 2.1612 ]]

In [59]:
a

[[1.1269, 1.183, 1.2243, 1.2492, 1.2788, 1.2965],
 [1.1824, 1.2859, 1.3613, 1.4135, 1.4672, 1.5115],
 [1.2294, 1.3579, 1.4684, 1.5566, 1.6217, 1.6898],
 [1.2524, 1.4258, 1.5346, 1.6621, 1.7622, 1.8395],
 [1.2841, 1.4762, 1.616, 1.7557, 1.8743, 2.0174],
 ['?', '?', '?', '?', '?', 2.1612]]